In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from ecephys_analyses.data import paths
from ecephys_analyses.sorting.postprocessing import run_postprocessing

In [ ]:
## USER

ks_sorting_condition = 'ks2_5_catgt_Th=12-10_lam=50_8s-batches'

postprocessing_condition = 'postpro_1'

##########
## Manual:
# Subject, condition, catgt_data
data_conditions = [
#     ('Eugene', '10-9-2020_NREM_depth1.5_imec0', True),
#     ('Eugene', '10-19-2020_NREM_depth1.5_imec0', True),
]
##########
## Auto

condition_groups = [
    ('Allan', 'SD'),
    ('Doppio', 'SD'),
    ('Luigi', 'SD'),
]

data_conditions = []
for subject, condition_group in condition_groups:
    data_conditions += [
        (subject, cond)
        for cond in paths.get_conditions(subject, condition_group)
    ]
print(f'Data condition N={len(data_conditions)} (Looks good?): {data_conditions}')

#########


n_jobs = 1  # Don't parallelize (possibly messed up tmp directories?)
assert n_jobs == 1

## end USER

In [ ]:
for (subject, condition) in data_conditions:
    print(subject, condition, ks_sorting_condition, postprocessing_condition)
    run_postprocessing(
        subject,
        condition,
        ks_sorting_condition,
        postprocessing_condition,
        rerun_existing=False
    )